# 필요 패키지 불러오기

In [266]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as BS
from datetime import datetime

In [187]:
mos_2015 = pd.read_csv("./서울시_모기_수집_데이터(2015).csv",encoding='euc-kr').iloc[1:,[0,2]].rename(columns={"DMS 포집내역 (2015.4.6 ~ 2015.10.31)":"date","Unnamed: 2":"mosquito_Indicator"})[25:].reset_index(drop=True)
mos_2016 = pd.read_csv("./서울시_모기_수집_데이터(2016).csv",encoding='euc-kr').iloc[1:,[0,2]].rename(columns={"DMS 포집내역 (2016.4.18 ~ 2016.10.31)":"date","Unnamed: 2":"mosquito_Indicator"})[13:].reset_index(drop=True)
mos_2017 = pd.read_csv("./서울시_모기_수집_데이터(2017).csv",encoding='euc-kr').iloc[1:,[0,2]].rename(columns={"DMS 포집내역 (2017.4.17 ~ 2017.10.31)":"date","Unnamed: 2":"mosquito_Indicator"})[14:].reset_index(drop=True)
mos_2018 = pd.read_csv("./서울시_모기_수집_데이터(2018).csv",encoding='euc-kr').iloc[1:,[0,2]].rename(columns={"DMS 포집내역 (2018.4.30 ~ 2018.10.31)":"date","Unnamed: 2":"mosquito_Indicator"})[1:].reset_index(drop=True)
mos_2019 = pd.read_csv("./서울시_모기_수집_데이터(2019).csv",encoding='euc-kr').iloc[1:,[0,2]].rename(columns={"DMS 포집내역 (2019.4.15 ~ 2019.10.31)":"date","Unnamed: 2":"mosquito_Indicator"})[16:].reset_index(drop=True)
mos_2020 = pd.read_csv("./서울시_모기_수집_데이터(2020).csv",encoding='euc-kr').iloc[1:,[0,2]].rename(columns={"DMS 포집내역 (2020.5.1 ~ 2020.10.31)":"date","Unnamed: 2":"mosquito_Indicator"})[:184]
mos_2021 = pd.read_csv("./서울시_모기_수집_데이터(2021).csv",encoding='euc-kr').iloc[1:,[0,2]].rename(columns={"DMS 포집내역 (2021.5.1 ~ 2021.10.31)":"date","Unnamed: 2":"mosquito_Indicator"})
mos_2022 = pd.read_csv("./서울시_모기_수집_데이터(2022).csv",encoding='euc-kr').iloc[1:,[0,2]].rename(columns={"DMS 포집내역 (2022.5.1 ~ 2022.10.31)":"date","Unnamed: 2":"mosquito_Indicator"})

In [80]:
mos_2015[mos_2015['mosquito_Indicator']==' - ']

,date,mosquito_Indicator
8,2015-05-09,-
9,2015-05-10,-
10,2015-05-11,-
80,2015-07-20,-
136,2015-09-14,-
137,2015-09-15,-


In [ ]:
mos_2021['mosquito_Indicator'] = mos_2021['mosquito_Indicator'].apply(lambda x : str(x) if type(x) == float else x)

In [190]:
mos_2021

,date,mosquito_Indicator
1,2021-05-01,388
2,2021-05-02,151
3,2021-05-03,608
4,2021-05-04,869
5,2021-05-05,448
...,...,...
180,2021-10-27,669
181,2021-10-28,691
182,2021-10-29,765
183,2021-10-30,829


In [192]:
mos_2015['mosquito_Indicator'] = mos_2015['mosquito_Indicator'].apply(lambda x : x.strip()).apply(lambda x : x.replace(x,'999999') if x == 'nan' or x == '-' else x)
mos_2016['mosquito_Indicator'] = mos_2016['mosquito_Indicator'].apply(lambda x : x.strip()).apply(lambda x : x.replace(x,'999999') if x == 'nan' or x == '-' else x)
mos_2017['mosquito_Indicator'] = mos_2017['mosquito_Indicator'].apply(lambda x : x.strip()).apply(lambda x : x.replace(x,'999999') if x == 'nan' or x == '-' else x)
mos_2018['mosquito_Indicator'] = mos_2018['mosquito_Indicator'].apply(lambda x : x.strip()).apply(lambda x : x.replace(x,'999999') if x == 'nan' or x == '-' else x)
mos_2019['mosquito_Indicator'] = mos_2019['mosquito_Indicator'].apply(lambda x : x.strip()).apply(lambda x : x.replace(x,'999999') if x == 'nan' or x == '-' else x)
mos_2020['mosquito_Indicator'] = mos_2020['mosquito_Indicator'].apply(lambda x : x.strip()).apply(lambda x : x.replace(x,'999999') if x == 'nan' or x == '-' else x)
mos_2021['mosquito_Indicator'] = mos_2021['mosquito_Indicator'].apply(lambda x : x.strip()).apply(lambda x : x.replace(x,'999999') if x == 'nan' or x == '-' else x)
mos_2022['mosquito_Indicator'] = mos_2022['mosquito_Indicator'].apply(lambda x : x.strip()).apply(lambda x : x.replace(x,'999999') if x == 'nan' or x == '-' else x)

# 연도별 결측치 개수

In [193]:
print(sum(mos_2015['mosquito_Indicator']=='999999'),sum(mos_2016['mosquito_Indicator']=='999999'),sum(mos_2017['mosquito_Indicator']=='999999'),sum(mos_2018['mosquito_Indicator']=='999999'),sum(mos_2019['mosquito_Indicator']=='999999'),sum(mos_2020['mosquito_Indicator']=='999999'),sum(mos_2021['mosquito_Indicator']=='999999'),sum(mos_2022['mosquito_Indicator']=='999999'))

6 5 0 0 0 0 14 0


# 연도별 강수량

In [292]:
# 2015년 서울 일 단위 강수량
url = "https://data.kma.go.kr/stcs/grnd/grndRnDay.do?pgmNo=156"
payload={"rnElement" : "RN", 
"fileType" : "", 
"pgmNo" : "156", 
"menuNo" : "552", 
"pageIndex" : "", 
"stnGroupSns" : "", 
"serviceSe" : "F00101", 
"selectType" : "1", 
"mddlClssCd" : "SFC01", 
"firstLoading" : "N", 
"dataFormCd" : "F00512", 
"schType" : "2", 
"startYear" : "2015", 
"endYear" : "2015", 
"startMonth" : "01", 
"endMonth" : "01", 
"schStnId" : "108", 
"txtStnNm" : "서울", 
"gFontSize" : ""}
r = requests.post(url,data=payload)
bs = BS(r.text)
table = str(bs.findAll('table', class_='tbl')[3])
df_2015 = pd.read_html(table)[0]
df_2015['날짜'] = df_2015['날짜'].apply(lambda x : '2015-0' + str(x) if len(str(x))== 3 else '2015-'+str(x)) 
df_2015['날짜'] = df_2015['날짜'].apply(lambda x : datetime.strptime(x,'%Y-%m%d').strftime('%Y-%m-%d'))
df_2015

,연도,날짜,지점,관측값
0,2015,2015-01-05,서울,0.4
1,2015,2015-01-06,서울,3.0
2,2015,2015-01-16,서울,0.3
3,2015,2015-01-18,서울,4.0
4,2015,2015-01-19,서울,0.1
...,...,...,...,...
99,2015,2015-12-15,서울,0.1
100,2015,2015-12-21,서울,2.0
101,2015,2015-12-23,서울,0.4
102,2015,2015-12-26,서울,0.7


In [283]:
# 2016년 서울 일 단위 강수량
url = "https://data.kma.go.kr/stcs/grnd/grndRnDay.do?pgmNo=156"
payload={"rnElement" : "RN", 
"fileType" : "", 
"pgmNo" : "156", 
"menuNo" : "552", 
"pageIndex" : "", 
"stnGroupSns" : "", 
"serviceSe" : "F00101", 
"selectType" : "1", 
"mddlClssCd" : "SFC01", 
"firstLoading" : "N", 
"dataFormCd" : "F00512", 
"schType" : "2", 
"startYear" : "2016", 
"endYear" : "2016", 
"startMonth" : "01", 
"endMonth" : "01", 
"schStnId" : "108", 
"txtStnNm" : "서울", 
"gFontSize" : ""}
r = requests.post(url,data=payload)
bs = BS(r.text)
table = str(bs.findAll('table', class_='tbl')[3])
df_2016 = pd.read_html(table)[0]
df_2016['날짜'] = df_2016['날짜'].apply(lambda x : '2016-0' + str(x) if len(str(x))== 3 else '2016-'+str(x)) 
df_2016['날짜'] = df_2016['날짜'].apply(lambda x : datetime.strptime(x,'%Y-%m%d').strftime('%Y-%m-%d'))
df_2016

,연도,날짜,지점,관측값
0,2016,2016-01-13,서울,0.4
1,2016,2016-01-16,서울,0.1
2,2016,2016-01-18,서울,0.2
3,2016,2016-01-26,서울,0.3
4,2016,2016-02-11,서울,0.1
...,...,...,...,...
104,2016,2016-12-21,서울,29.0
105,2016,2016-12-22,서울,23.5
106,2016,2016-12-23,서울,0.1
107,2016,2016-12-27,서울,0.5


In [284]:
# 2017년 서울 일 단위 강수량
url = "https://data.kma.go.kr/stcs/grnd/grndRnDay.do?pgmNo=156"
payload={"rnElement" : "RN", 
"fileType" : "", 
"pgmNo" : "156", 
"menuNo" : "552", 
"pageIndex" : "", 
"stnGroupSns" : "", 
"serviceSe" : "F00101", 
"selectType" : "1", 
"mddlClssCd" : "SFC01", 
"firstLoading" : "N", 
"dataFormCd" : "F00512", 
"schType" : "2", 
"startYear" : "2017", 
"endYear" : "2017", 
"startMonth" : "01", 
"endMonth" : "01", 
"schStnId" : "108", 
"txtStnNm" : "서울", 
"gFontSize" : ""}
r = requests.post(url,data=payload)
bs = BS(r.text)
table = str(bs.findAll('table', class_='tbl')[3])
df_2017 = pd.read_html(table)[0]
df_2017['날짜'] = df_2017['날짜'].apply(lambda x : '2017-0' + str(x) if len(str(x))== 3 else '2017-'+str(x)) 
df_2017['날짜'] = df_2017['날짜'].apply(lambda x : datetime.strptime(x,'%Y-%m%d').strftime('%Y-%m-%d'))
df_2017

,연도,날짜,지점,관측값
0,2017,2017-01-02,서울,0.3
1,2017,2017-01-13,서울,0.6
2,2017,2017-01-20,서울,4.0
3,2017,2017-01-21,서울,2.1
4,2017,2017-01-26,서울,0.3
...,...,...,...,...
97,2017,2017-12-18,서울,3.4
98,2017,2017-12-20,서울,0.2
99,2017,2017-12-24,서울,20.0
100,2017,2017-12-30,서울,0.7


In [285]:
# 2018년 서울 일 단위 강수량
url = "https://data.kma.go.kr/stcs/grnd/grndRnDay.do?pgmNo=156"
payload={"rnElement" : "RN", 
"fileType" : "", 
"pgmNo" : "156", 
"menuNo" : "552", 
"pageIndex" : "", 
"stnGroupSns" : "", 
"serviceSe" : "F00101", 
"selectType" : "1", 
"mddlClssCd" : "SFC01", 
"firstLoading" : "N", 
"dataFormCd" : "F00512", 
"schType" : "2", 
"startYear" : "2018", 
"endYear" : "2018", 
"startMonth" : "01", 
"endMonth" : "01", 
"schStnId" : "108", 
"txtStnNm" : "서울", 
"gFontSize" : ""}
r = requests.post(url,data=payload)
bs = BS(r.text)
table = str(bs.findAll('table', class_='tbl')[3])
df_2018 = pd.read_html(table)[0]
df_2018['날짜'] = df_2018['날짜'].apply(lambda x : '2018-0' + str(x) if len(str(x))== 3 else '2018-'+str(x)) 
df_2018['날짜'] = df_2018['날짜'].apply(lambda x : datetime.strptime(x,'%Y-%m%d').strftime('%Y-%m-%d'))
df_2018

,연도,날짜,지점,관측값
0,2018,2018-01-08,서울,0.9
1,2018,2018-01-09,서울,0.5
2,2018,2018-01-10,서울,0.3
3,2018,2018-01-13,서울,0.4
4,2018,2018-01-15,서울,0.2
...,...,...,...,...
99,2018,2018-12-03,서울,9.5
100,2018,2018-12-04,서울,5.0
101,2018,2018-12-13,서울,1.3
102,2018,2018-12-16,서울,0.3


In [286]:
# 2019년 서울 일 단위 강수량
url = "https://data.kma.go.kr/stcs/grnd/grndRnDay.do?pgmNo=156"
payload={"rnElement" : "RN", 
"fileType" : "", 
"pgmNo" : "156", 
"menuNo" : "552", 
"pageIndex" : "", 
"stnGroupSns" : "", 
"serviceSe" : "F00101", 
"selectType" : "1", 
"mddlClssCd" : "SFC01", 
"firstLoading" : "N", 
"dataFormCd" : "F00512", 
"schType" : "2", 
"startYear" : "2019", 
"endYear" : "2019", 
"startMonth" : "01", 
"endMonth" : "01", 
"schStnId" : "108", 
"txtStnNm" : "서울", 
"gFontSize" : ""}
r = requests.post(url,data=payload)
bs = BS(r.text)
table = str(bs.findAll('table', class_='tbl')[3])
df_2019 = pd.read_html(table)[0]
df_2019['날짜'] = df_2019['날짜'].apply(lambda x : '2019-0' + str(x) if len(str(x))== 3 else '2019-'+str(x)) 
df_2019['날짜'] = df_2019['날짜'].apply(lambda x : datetime.strptime(x,'%Y-%m%d').strftime('%Y-%m-%d'))
df_2019

,연도,날짜,지점,관측값
0,2019,2019-02-01,서울,0.3
1,2019,2019-02-03,서울,16.0
2,2019,2019-02-07,서울,0.5
3,2019,2019-02-15,서울,2.2
4,2019,2019-02-16,서울,0.2
...,...,...,...,...
93,2019,2019-12-22,서울,0.3
94,2019,2019-12-23,서울,2.9
95,2019,2019-12-26,서울,0.3
96,2019,2019-12-29,서울,1.4


In [288]:
# 2020년 서울 일 단위 강수량
url = "https://data.kma.go.kr/stcs/grnd/grndRnDay.do?pgmNo=156"
payload={"rnElement" : "RN", 
"fileType" : "", 
"pgmNo" : "156", 
"menuNo" : "552", 
"pageIndex" : "", 
"stnGroupSns" : "", 
"serviceSe" : "F00101", 
"selectType" : "1", 
"mddlClssCd" : "SFC01", 
"firstLoading" : "N", 
"dataFormCd" : "F00512", 
"schType" : "2", 
"startYear" : "2020", 
"endYear" : "2020", 
"startMonth" : "01", 
"endMonth" : "01", 
"schStnId" : "108", 
"txtStnNm" : "서울", 
"gFontSize" : ""}
r = requests.post(url,data=payload)
bs = BS(r.text)
table = str(bs.findAll('table', class_='tbl')[3])
df_2020 = pd.read_html(table)[0]
df_2020['날짜'] = df_2020['날짜'].apply(lambda x : '2020-0' + str(x) if len(str(x))== 3 else '2020-'+str(x)) 
df_2020['날짜'] = df_2020['날짜'].apply(lambda x : datetime.strptime(x,'%Y-%m%d').strftime('%Y-%m-%d'))
df_2020

,연도,날짜,지점,관측값
0,2020,2020-01-01,서울,0.1
1,2020,2020-01-06,서울,1.8
2,2020,2020-01-07,서울,46.3
3,2020,2020-01-08,서울,11.6
4,2020,2020-01-19,서울,0.6
...,...,...,...,...
101,2020,2020-11-22,서울,4.8
102,2020,2020-12-13,서울,2.9
103,2020,2020-12-18,서울,0.2
104,2020,2020-12-28,서울,1.3


In [289]:
# 2021년 서울 일 단위 강수량
url = "https://data.kma.go.kr/stcs/grnd/grndRnDay.do?pgmNo=156"
payload={"rnElement" : "RN", 
"fileType" : "", 
"pgmNo" : "156", 
"menuNo" : "552", 
"pageIndex" : "", 
"stnGroupSns" : "", 
"serviceSe" : "F00101", 
"selectType" : "1", 
"mddlClssCd" : "SFC01", 
"firstLoading" : "N", 
"dataFormCd" : "F00512", 
"schType" : "2", 
"startYear" : "2021", 
"endYear" : "2021", 
"startMonth" : "01", 
"endMonth" : "01", 
"schStnId" : "108", 
"txtStnNm" : "서울", 
"gFontSize" : ""}
r = requests.post(url,data=payload)
bs = BS(r.text)
table = str(bs.findAll('table', class_='tbl')[3])
df_2021 = pd.read_html(table)[0]
df_2021['날짜'] = df_2021['날짜'].apply(lambda x : '2021-0' + str(x) if len(str(x))== 3 else '2021-'+str(x)) 
df_2021['날짜'] = df_2021['날짜'].apply(lambda x : datetime.strptime(x,'%Y-%m%d').strftime('%Y-%m-%d'))
df_2021

,연도,날짜,지점,관측값
0,2021,2021-01-06,서울,2.3
1,2021,2021-01-12,서울,2.6
2,2021,2021-01-15,서울,0.2
3,2021,2021-01-18,서울,0.3
4,2021,2021-01-21,서울,9.2
...,...,...,...,...
112,2021,2021-12-15,서울,2.1
113,2021,2021-12-18,서울,2.8
114,2021,2021-12-19,서울,0.2
115,2021,2021-12-24,서울,0.2


In [290]:
# 2022년 서울 일 단위 강수량
url = "https://data.kma.go.kr/stcs/grnd/grndRnDay.do?pgmNo=156"
payload={"rnElement" : "RN", 
"fileType" : "", 
"pgmNo" : "156", 
"menuNo" : "552", 
"pageIndex" : "", 
"stnGroupSns" : "", 
"serviceSe" : "F00101", 
"selectType" : "1", 
"mddlClssCd" : "SFC01", 
"firstLoading" : "N", 
"dataFormCd" : "F00512", 
"schType" : "2", 
"startYear" : "2022", 
"endYear" : "2022", 
"startMonth" : "01", 
"endMonth" : "01", 
"schStnId" : "108", 
"txtStnNm" : "서울", 
"gFontSize" : ""}
r = requests.post(url,data=payload)
bs = BS(r.text)
table = str(bs.findAll('table', class_='tbl')[3])
df_2022 = pd.read_html(table)[0]
df_2022['날짜'] = df_2022['날짜'].apply(lambda x : '2022-0' + str(x) if len(str(x))== 3 else '2022-'+str(x)) 
df_2022['날짜'] = df_2022['날짜'].apply(lambda x : datetime.strptime(x,'%Y-%m%d').strftime('%Y-%m-%d'))
df_2022

,연도,날짜,지점,관측값
0,2022,2022-01-02,서울,0.3
1,2022,2022-01-10,서울,0.9
2,2022,2022-01-11,서울,0.2
3,2022,2022-01-17,서울,0.5
4,2022,2022-01-19,서울,1.7
...,...,...,...,...
99,2022,2022-12-13,서울,0.6
100,2022,2022-12-15,서울,2.4
101,2022,2022-12-17,서울,0.2
102,2022,2022-12-21,서울,3.6


In [293]:
df_2015 = df_2015.iloc[:,[1,3]].rename(columns={'날짜':'date','관측값':'rain(mm)'})
df_2016 = df_2016.iloc[:,[1,3]].rename(columns={'날짜':'date','관측값':'rain(mm)'})
df_2017 = df_2017.iloc[:,[1,3]].rename(columns={'날짜':'date','관측값':'rain(mm)'})
df_2018 = df_2018.iloc[:,[1,3]].rename(columns={'날짜':'date','관측값':'rain(mm)'})
df_2019 = df_2019.iloc[:,[1,3]].rename(columns={'날짜':'date','관측값':'rain(mm)'})
df_2020 = df_2020.iloc[:,[1,3]].rename(columns={'날짜':'date','관측값':'rain(mm)'})
df_2021 = df_2021.iloc[:,[1,3]].rename(columns={'날짜':'date','관측값':'rain(mm)'})
df_2022 = df_2022.iloc[:,[1,3]].rename(columns={'날짜':'date','관측값':'rain(mm)'})

In [311]:
df_2022.iloc[89,:]

date        2022-10-09
rain(mm)          20.4
Name: 89, dtype: object

# 연도별 모기 관측수 + 강수량

In [343]:
mos_rain_2015 = pd.merge(mos_2015,df_2015,on='date',how='left').fillna(0.0)
mos_rain_2016 = pd.merge(mos_2016,df_2016,on='date',how='left').fillna(0.0)
mos_rain_2017 = pd.merge(mos_2017,df_2017,on='date',how='left').fillna(0.0)
mos_rain_2018 = pd.merge(mos_2018,df_2018,on='date',how='left').fillna(0.0)
mos_rain_2019 = pd.merge(mos_2019,df_2019,on='date',how='left').fillna(0.0)
mos_rain_2020 = pd.merge(mos_2020,df_2020,on='date',how='left').fillna(0.0)
mos_rain_2021 = pd.merge(mos_2021,df_2021,on='date',how='left').fillna(0.0)
mos_rain_2022 = pd.merge(mos_2022,df_2022,on='date',how='left').fillna(0.0)

# 연도별 서울 온도 데이터 (일단위)

In [374]:
temp_2015 = pd.read_csv("./ta_20230824201440.csv").iloc[:,[0,2,3,4]].rename(columns={"��¥":"date","��ձ��(��)":"mean_T(℃)","�������(��)":"min_T(℃)","�ְ����(��)":"max_T(℃)"})
temp_2016 = pd.read_csv("./ta_20230824202813.csv").iloc[:,[0,2,3,4]].rename(columns={"��¥":"date","��ձ��(��)":"mean_T(℃)","�������(��)":"min_T(℃)","�ְ����(��)":"max_T(℃)"})
temp_2017 = pd.read_csv("./ta_20230824203123.csv").iloc[:,[0,2,3,4]].rename(columns={"��¥":"date","��ձ��(��)":"mean_T(℃)","�������(��)":"min_T(℃)","�ְ����(��)":"max_T(℃)"})
temp_2018 = pd.read_csv("./ta_20230824203300.csv").iloc[:,[0,2,3,4]].rename(columns={"��¥":"date","��ձ��(��)":"mean_T(℃)","�������(��)":"min_T(℃)","�ְ����(��)":"max_T(℃)"})
temp_2019 = pd.read_csv("./ta_20230824203416.csv").iloc[:,[0,2,3,4]].rename(columns={"��¥":"date","��ձ��(��)":"mean_T(℃)","�������(��)":"min_T(℃)","�ְ����(��)":"max_T(℃)"})
temp_2020 = pd.read_csv("./ta_20230824203532.csv").iloc[:,[0,2,3,4]].rename(columns={"��¥":"date","��ձ��(��)":"mean_T(℃)","�������(��)":"min_T(℃)","�ְ����(��)":"max_T(℃)"})
temp_2021 = pd.read_csv("./ta_20230824203639.csv").iloc[:,[0,2,3,4]].rename(columns={"��¥":"date","��ձ��(��)":"mean_T(℃)","�������(��)":"min_T(℃)","�ְ����(��)":"max_T(℃)"})
temp_2022 = pd.read_csv("./ta_20230824203732.csv").iloc[:,[0,2,3,4]].rename(columns={"��¥":"date","��ձ��(��)":"mean_T(℃)","�������(��)":"min_T(℃)","�ְ����(��)":"max_T(℃)"})

# 연도별 최종 데이터 (날짜, 모기 관측 수, 강수량, 평균기온, 최저기온, 최고기온)

In [378]:
total_2015 = pd.merge(mos_rain_2015,temp_2015,on='date')
total_2016 = pd.merge(mos_rain_2016,temp_2016,on='date')
total_2017 = pd.merge(mos_rain_2017,temp_2017,on='date')
total_2018 = pd.merge(mos_rain_2018,temp_2018,on='date')
total_2019 = pd.merge(mos_rain_2019,temp_2019,on='date')
total_2020 = pd.merge(mos_rain_2020,temp_2020,on='date')
total_2021 = pd.merge(mos_rain_2021,temp_2021,on='date')
total_2022 = pd.merge(mos_rain_2022,temp_2022,on='date')

In [389]:
total_2015['mosquito_Indicator'] = total_2015['mosquito_Indicator'].apply(lambda x : np.nan if x == '999999' else x)
total_2016['mosquito_Indicator'] = total_2016['mosquito_Indicator'].apply(lambda x : np.nan if x == '999999' else x)
total_2017['mosquito_Indicator'] = total_2017['mosquito_Indicator'].apply(lambda x : np.nan if x == '999999' else x)
total_2018['mosquito_Indicator'] = total_2018['mosquito_Indicator'].apply(lambda x : np.nan if x == '999999' else x)
total_2019['mosquito_Indicator'] = total_2019['mosquito_Indicator'].apply(lambda x : np.nan if x == '999999' else x)
total_2020['mosquito_Indicator'] = total_2020['mosquito_Indicator'].apply(lambda x : np.nan if x == '999999' else x)
total_2021['mosquito_Indicator'] = total_2021['mosquito_Indicator'].apply(lambda x : np.nan if x == '999999' else x)
total_2022['mosquito_Indicator'] = total_2022['mosquito_Indicator'].apply(lambda x : np.nan if x == '999999' else x)

In [394]:
total_2017[total_2017['max_T(℃)'].isnull()]

,date,mosquito_Indicator,rain(mm),mean_T(℃),min_T(℃),max_T(℃)
164,2017-10-12,"1,610",0.0,11.4,8.8,NaN
